<a id='intro'></a>

# Tools for exploring ICOS Atmospheric data & STILT results


This notebook, which is supposed to me embedded into a calling notebook, includes functions in Python for exploring ICOS data and STILT results. The notebook is divided in the following parts:

- [Import Python modules](#import_modules)
- [Global variables](#global_vars)
- [Data availability](#data_avail)
- [Map functions](#map_funcs)
- [Plotting functions](#plotting_funcs)
- [Footprint functions](#footprint_funcs)
- [Widget functions](#widget_func)




Use the links to quickly navigate to the parts you are interested in. 

<a id='import_modules'></a>
<br>
<br>

## 1 Import Python modules
In this part, we are going to import all Python modules that we are going to use.

In [ ]:
#Import modules:
import sys
import os

import numpy as np
from numpy import nan
import pandas as pd

import pickle
import zipfile
import netCDF4 as cdf
from datetime import datetime, date 
import datetime as dt
from dateutil.relativedelta import relativedelta
import requests
import fnmatch

from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.markers as markers

import folium
import branca


import cartopy
cartopy.config['data_dir'] = '/data/project/cartopy/'
from cartopy import config
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
import cartopy.feature as cfeature


from ipywidgets import interact, interact_manual, ColorPicker, Dropdown, SelectMultiple, Checkbox, DatePicker, SelectionRangeSlider, HTML

#Import modules to create figure:
from bokeh.plotting import figure, show
from bokeh.io import reset_output, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker, Title


# Import notebook scripts
from tools.check_funcs.numeric import is_int
from tools.country.fullname import get_country_fullname_from_iso3166_2char
from tools.visualization.string.format_output import printmd
from tools.visualization.bokeh_help_funcs.secondary_yaxis import set_yranges_2y
from tools.visualization.availability import plot_attr, availability_plot
from tools.math.roundfuncs import rounddown_10, rounddown_20, rounddown_100, roundup_10, roundup_20, roundup_100

   
#Import ICOS tools:
from icoscp.sparql import sparqls, runsparql
from icoscp.cpb.dobj import Dobj
import icoscp.const as CPC
from icoscp.station import station
from icoscp.stilt import stiltstation


#Set the notebook as the default output location:
reset_output()
output_notebook()


<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='global_vars'></a>

<br>
<br>

## 2. Global variables
In this section, of the notebook, we set variables that are global within the notebook in order to explain the purpose once and for all. 

In [ ]:
# The ICOS pylib use icoscp.const, imported above as CPC, for static links and local paths to data.
# In particular, paths are LOCAL to the ICOS Carbon Portal server, and hence usage of paths from CPC are 
# only expected to run on the ICOS Carbon Portal server like the jupyter environment https://exploredata.icos-cp.eu/

# STILTFP is the local path to STILT foot print data
STILTFP = CPC.STILTFP

# This is a link to a csv file which is a table mapping STILT-stations to ICOS-stations, 
# details are found in the function get_stilt_icos_compare_df(). 
STILTINFO = CPC.STILTINFO

# We will frequently use the list of ICOS atmospheric measurement stations:
atm_stations = station.getList(['as'])

# The availability table will be populated the first time function 
# get_timeseries_availability_stilt_icos() is called (in this way it will 
# not matter in what order the user runs the calling notebook) 
global availability_df     # This line is not needed, but can be helpful
availability_df = None     # This line is essential

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='data_avail'></a>

<br>
<br>

## 3. Station data
In this section we produce pandas dataframes for 
- Availability of CO$_2$ atmospheric data per ICOS station (Level 1 & Level 2)
- Availability of STILT data per station
- Availability of both STILT and ICOS data (Level 1 & Level 2 CO$_2$ atmospheric data products) 
- Positions of ICOS stations with STILT data
             


In [ ]:
def get_stilt_icos_compare_df():
    """
    Description:    Returns pandas.DataFrame with stiltstations corresponding to ICOS stations.
    Disclaimer:     Here we load a csv-file, as at the time of writing, is found at 
                    https://stilt.icos-cp.eu/viewer/stationinfo  (CPC.STILTINFO)
                    the user may notice that there are discrepancies between 
                    the ICOS sampling heights and the corresponding STILT altitudes, 
                    one reason for this is that STILT calculations are mean value 
                    calculations over a region of 10 km^2.
                    
                    We will use this information to filter out exceptions from the 
                    STILT-stations collected in the function get_stilt_of_icos_df()  
    """ 
    
    df = pd.read_csv(STILTINFO).dropna()
    
    return df
    
    
def get_stilt_of_icos_df():
    """
    Description: Returns pandas.DataFrame with stiltation data at ICOS stations. 
    """ 

    # Using the find function of the stiltstation class we get plenty of information on 
    # the stilt data, including years and months where there is data.
    # Here we set:
    #    - progress = False, in order to avoid an extra progress bar while loading the data.
    #    - project = 'icos', because we are only interested in STILT calculations located at 
    #      physical ICOS stations 
    #    - outfmt='pandas', in order to get a DataFrame
    stiltStations_df = stiltstation.find(progress = False, project='icos', outfmt='pandas')
    
    # Next, we clean the output in order to make it easier to compare to the measured ICOS-data
    #      - the format of data in the column 'alt' (altitude) of the STILT-data are integers. 
    stiltStations_df['Station'] = stiltStations_df.id.str[:3]

    #      - we remove some columns we will not use
    #      - finally we rename the columns so that it is clear what columns belong to STILT-data 
    stiltStations_df.reset_index(inplace = True)
    stiltStations_df.drop(['index','lat','lon','geoinfo'], axis=1, inplace = True)
    
    
    col_id_and_location = (lambda x: 'stiltStationId' if x=='id' else x)
    col_names = (lambda x: 'stilt' + x[0].upper() + x[1:] if x not in ['id','locIdent'] else col_id_and_location(x))
    stiltStations_df.rename(columns = col_names, inplace = True)
    
    return stiltStations_df


def get_icos_station_df():
    """
    Description: Returns pandas.DataFrame with main keys to label 1 and label 2 
                 atmospheric data to CO2 timeseries of ICOS stations.
    """ 

    label1 = 'ICOS ATC NRT CO2 growing time series'
    label2 = 'ICOS ATC CO2 Release'
    
    station_ls = list()
    for st in atm_stations:
        df = st.data()
        df = df.loc[(df.specLabel.isin([label1,label2]))]
        station_ls.append(df)

    icos_df = pd.concat(station_ls)
    
    #Reset index:
    icos_df.reset_index(inplace=True)

    #Add column with ICOS station ID:
    icos_df['stationId'] = [icos_df.station.iloc[y][-3:] for y in range(len(icos_df))]

    #Drop old integer index:
    icos_df.drop(['index'], axis=1, inplace=True)

    #Return dataframe:
    return icos_df


def get_timeseries_availability_stilt_icos():
    """
    Description: Returns pandas.DataFrame with availability of CO2-timeseries data of 
                    - ICOS label 1 
                    - ICOS label 2
                    - STILT calculations
    """ 
    # In order to set the global variable, we need to tell Python
    # that this variable is not local
    global availability_df 
    
    if isinstance(availability_df,pd.DataFrame) and not availability_df.empty:
        return availability_df

    # ICOS data
    icos_df = get_icos_station_df()

    # Main STILT-info
    stilt_df = get_stilt_of_icos_df()
 
    
    # In principle we will merge the two dataframes icos_df and stilt_df. 
    # However, in order to do this we need to make sure the data is comparable, 
    # in particular the "STILT altitude" is not identical to the "ICOS samplingheigt" 
    # (for more details, we refer to the description of the function get_stilt_icos_compare_df() above).    
    # We will now
    #   - filter the STILT data using the comparison table from get_stilt_icos_compare_df(), 
    #   - add the ICOS sampling height into as a new columns 'stiltIcosHeight' of the STILT dataframe  
    #     and change the format of this column from 'float' to 'str'
    stilt_filter_df = get_stilt_icos_compare_df()
    stilt_filter_df.drop(['STILT name','ICOS id','Country','STILT lat','STILT lon', 'STILT alt'], axis=1, inplace=True)
    stilt_filter_df.rename(columns = {'STILT id': 'stiltStationId' , 'ICOS height': 'stiltIcosHeight'}, inplace=True)
    stilt_filter_df.stiltIcosHeight = stilt_filter_df.stiltIcosHeight.astype(str)
    stilt_df = stilt_df.merge(stilt_filter_df, how='inner', on='stiltStationId')
    
    # We are now in position to merge the dataframes 
    availability_stilt_icos_df = pd.merge(icos_df,stilt_df, 
                                          left_on = ['stationId', 'samplingheight'], 
                                          right_on = ['stiltStation', 'stiltIcosHeight'])
    availability_stilt_icos_df.drop(['stiltStation', 'stiltIcosHeight'], axis=1, inplace=True)
    
    # Finally we convert the data type of columns with time-info from string to datetime:
    availability_stilt_icos_df['timeStart'] = pd.to_datetime(availability_stilt_icos_df['timeStart']) 
    availability_stilt_icos_df['timeEnd'] = pd.to_datetime(availability_stilt_icos_df['timeEnd']) 
    availability_df = availability_stilt_icos_df
    return availability_stilt_icos_df

    

def get_position_table():
    avail_df = get_timeseries_availability_stilt_icos()
    
    # Get a dataframe with station info for all ICOS atmospheric stations:
    icos_st_info_df = pd.concat([pd.DataFrame(atm_stations[i].info()) for i in range(len(atm_stations))])

    # Filter out stations for which there are no STILT-results:
    station_info_df = icos_st_info_df[icos_st_info_df['stationId'].isin(avail_df['stationId'].unique())].reset_index().drop('index', axis=1)
    
    return station_info_df

In [ ]:
def list_of_availability_dataframes() -> list: 
    """
    Description: The function returns a list of four dataframes where each 
                 dataframe is adjusted to work with the availability plot functions of 
                 tools/visualization/availability/availability_plot.py
                 - dataframe no 1 holds availability of ICOS Level 1
                 - dataframe no 2 holds availability of ICOS Level 2
                 - dataframe no 3 holds availability of STILT data
                 - dataframe no 4 holds availability of both STILT and ICOS Level 1 data
                 - dataframe no 5 holds availability of both STILT and ICOS Level 2 data
                 Note 1: There are no common dates for ICOS Level 1 and ICOS Level 2 data. 
                 After quality checks the Level 1 data will be released, and at this moment 
                 it will get the ICOS Level 2 label. Hence, there is no dataframe with common 
                 dates for ICOS Level 1 and ICOS Level 2 data.
                 Note 2: The calculated STILT data use data from different sources, wich might 
                 imply there is no common dates for STILT data and ICOS Level 1 data.  
    """
    
    
    def icos_df_to_plot(icos_df):
        # This is just an auxilary function to produce plotable data frames for ICOS-availability 
        # Columns: stiltStationId, year, month, date,decimalDate
        def avail_per_station(start, end):
            # This is just a sub-function to produce years and months 
            start = pd.Timestamp(start)
            end = pd.Timestamp(end)

            y1 = start.year 
            m1 = start.month 

            y2 = end.year + 1
            m2 = end.month + 1

            years = []
            months = []
            for y in range(y1,y2):
                min_month = 1 if y > y1 else m1
                max_month = 13 if y < y2 - 1 else m2
                for m in range(min_month,max_month):
                    years.append(y)
                    months.append(m)
            return pd.DataFrame({'year': years, 'month': months}) 

        df_list = []
        for x in icos_df.index:
            df_stn = avail_per_station(icos_df['timeStart'].values[x], icos_df['timeEnd'].values[x])
            df_stn['stiltStationId'] = icos_df['stiltStationId'].values[x]
            df_list.append(df_stn)

        # Gather the dataframe in a large dataframe
        df = pd.concat(df_list)
        df['date'] = pd.to_datetime(df.year*10000 + df.month*100 + 1, format='%Y%m%d')
        df['decimalDate'] = df.year + (df.month - 1)/12      

        df.reset_index().drop('index', axis=1)
        return df
    
    def stilt_df_to_plot(stilt_df):
        # This is just an auxilary function to produce plotable data frames for STILT-availability 
        # Columns: stiltStationId, year, month, date,decimalDate
        years = []
        months = []
        df_list = []
        for x in stilt_df.T:
            for y in stilt_df.stiltYears.values[x]:
                for m in stilt_df['stilt' + y].values[x]['months']:
                    years.append(int(y))
                    months.append(int(m))
            df_stn = pd.DataFrame({'year': years, 'month': months})
            df_stn['stiltStationId'] = stilt_df['stiltStationId'].values[x]
            df_list.append(df_stn)
            years = []
            months  = []
        
        # Gather the dataframe in a large dataframe
        df = pd.concat(df_list)
        df['date'] = pd.to_datetime(df.year*10000 + df.month*100 + 1, format='%Y%m%d')
        df['decimalDate'] = df.year + (df.month - 1)/12      

        df.reset_index().drop('index', axis=1)
        return df

    avail_df = get_timeseries_availability_stilt_icos()
    

    ## 1. Dataframe for ICOS Level 1 CO2 atmospheric product: 
    co2_l1_label = 'ICOS ATC NRT CO2 growing time series'
    icos_L1_df = icos_df_to_plot(avail_df[['stiltStationId','timeStart','timeEnd']].loc[
        avail_df['specLabel']==co2_l1_label].reset_index())

    ## 2. Dataframe for ICOS Level 2 CO2 atmospheric product: 
    co2_l2_label = 'ICOS ATC CO2 Release'
    icos_L2_df = icos_df_to_plot(avail_df[['stiltStationId','timeStart','timeEnd']].loc[
        avail_df['specLabel']==co2_l2_label].reset_index())
    
    ## 3. Dataframe for STILT data: 
    stilt_df = avail_df[[x for x in avail_df if x[:5] == 'stilt']] 

    # Next we remove duplicate STILT-rows 
    # Note: Here we use a "mask" to remove rows, pandas will not allow 
    # drop_duplicates() to a dataframe containing lists
    stilt_df = stilt_df[~stilt_df.astype(str).duplicated()].reset_index().drop('index', axis=1)
    stilt_df = stilt_df_to_plot(stilt_df)    

    ## 4. The following table shows where there is both ICOS Level 2 data and STILT data
    icos_L1_stilt_intersect_df = pd.merge(icos_L1_df,
                                          stilt_df,
                                          how='inner',
                                          on=list(icos_L1_df.columns.values), sort=True)


    ## 5. The following table shows where there is both ICOS Level 2 data and STILT data
    icos_L2_stilt_intersect_df = pd.merge(icos_L2_df,
                                          stilt_df,
                                          how='inner',
                                          on=list(icos_L2_df.columns.values), sort=True)
   

    return [icos_L1_df, icos_L2_df, stilt_df, icos_L1_stilt_intersect_df, icos_L2_stilt_intersect_df]

In [ ]:
def update_availability_plot(): 
    
    
    # Dataframe of availability of timeseries
    stilt_icos_df = get_timeseries_availability_stilt_icos()
    
    # Create a new object of the class PlotAttr:
    avail_plot_attr = plot_attr.PlotAttr()

    # Provide list of tick-values for y-axis:
    avail_plot_attr.y_range=list(reversed(sorted((list(stilt_icos_df.reset_index().stiltStationId.unique())))))

    # Define plot title:
    avail_plot_attr.title_text = 'ICOS - STILT availability table'

    # Set x- & y-axis labels:
    avail_plot_attr.xaxis_label = 'Time'
    avail_plot_attr.yaxis_label = 'Station'

    # Provide dataframe column name for values that will be plotted on x- & y-axis:
    avail_plot_attr.col_name_xaxis = 'decimalDate'
    avail_plot_attr.col_name_yaxis = 'stiltStationId'
    
    # Get a list of dataframes with availability info for
    # every combination of ICOS L1, L2 & STILT data products
    # per station, year and month:
    data_set_ls = list_of_availability_dataframes()
     
    # List with labels for every non-empty dataframe to plot  
    layer_ls = ['ICOS L1', 'ICOS L2', 'STILT', 'STILT and ICOS L1', 'STILT and ICOS L2']
    
    # Remove empty dataframes and corresponding labels
    x=0
    while x < len(data_set_ls):
        if data_set_ls[x].empty:
            data_set_ls.pop(x)
            layer_ls.pop(x)
        else:
            x+=1
    
    t_list = [("Station", "stiltStationId"),
              ("Year", "year"),
              ("Month", "month")]
    
    # Call function to display plot:
    availability_plot.plot_table(data_set_ls, layer_ls, t_list, avail_plot_attr)

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='map_funcs'></a>

<br>
<br>

## 4. Map functions
This part includes functions that create a map of stations for which STILT results and ICOS CO$_2$ atmospheric data products are available for.


In [ ]:
def get_sampl_height_AS_CO2(station_code):
    # Return list of sampling heights for ICOS CO2 atmosphere data products:
    
    df = get_timeseries_availability_stilt_icos()
    
    return sorted([float(x) for x in set(df.loc[df.stationId == station_code].samplingheight)])

In [ ]:
def plotmap_stilt(stations_df, selected_station, basemap, d_icon='cloud', icon_col='orange'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Sun Sep 13 17:00:00 2020
    Last Changed:     Sun Sep 13 17:00:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a pandas dataframe with info about ICOS Stations 
                      (for wich STILT results are available for),
                      the 3-character long station code of a selected station, the basemap type, the
                      marker icon and the marker color as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: pandas dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)
                      3. Type of basemap (e.g. OSM or imagery)
                         (var_name: 'basemap', var_type: String)
                      4. Marker icon name (domain specific)
                         (var_name: 'd_icon', var_type: String)
                      5. Marker color (domain specific)
                         (var_name: 'icon_col', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
   
    #Check what type of basemap is selected:
    if(basemap=='Imagery'):
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
                       zoom_start=15,
                       tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                       attr = 'Esri',
                       name = 'Esri Satellite',
                       overlay = False,
                       control = True)

    else:
        
        #Create folium map-object:
        m = folium.Map(location=[float(stations_df.loc[stations_df.stationId==selected_station].lat.values[0]),
                                 float(stations_df.loc[stations_df.stationId==selected_station].lon.values[0])],
                       zoom_start=15)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, st_dict, marker_txt, marker_color):
        
        #Add popup text:
        popup=folium.Popup(marker_txt, parse_html=True, max_width=400)
        
        #Create marker and add it to the map:
        folium.Marker(location=[float(st_dict['lat']),float(st_dict['lon'])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon=d_icon),
                      tooltip=tooltip).add_to(map_obj)



    #Create markers for all stations except selected station:
    for st in range(len(stations_df)):
        
        #Create and initialize variable to store station info in html table:
        html_table = """<meta content="text/html; charset=UTF-8">
                        <style>td{padding: 3px;}</style><table>"""
        
        for i in range(len(stations_df.columns)):
            
            #Check if column contains info about 'country':
            if(stations_df.columns[i]=='country'):
                
                #Add country name:
                html_table = html_table+'<tr><td>'+ stations_df.columns[i]+': </td><td><b>'+get_country_fullname_from_iso3166_2char(str(stations_df.iloc[st][stations_df.columns[i]]))+'</b></td></tr>'
            
            
            #Check if column contains info about station PI:
            elif((stations_df.columns[i]=='lastName')& (('firstName' in stations_df.columns)&('lastName' in stations_df.columns))):
                
                #Add project info:
                html_table = html_table+'<tr><td>station PI: </td><td><b>'+str(stations_df.iloc[st].firstName)+' '+str(stations_df.iloc[st].lastName)+'</b></td></tr>'
        
        
            #Check if dataframe contains info about station name and station uri:
            elif((stations_df.columns[i]=='name') & ('uri' in stations_df.columns)):
                
                #Add link to station landing page:
                html_table = html_table+'<tr><td>'+stations_df.columns[i]+'</td><td><b><a href="'+str(stations_df.iloc[st]['uri'])+'"target="_blank">'+str(stations_df.iloc[st][stations_df.columns[i]])+'</a></b></td></tr>'           
            
            #Skip columns:
            elif((stations_df.columns[i]=='firstName') | (stations_df.columns[i]=='uri')):
                continue
            
            else:
                
                #Add column info:
                html_table = html_table+'<tr><td>'+ stations_df.columns[i]+': </td><td><b>'+str(stations_df.iloc[st][stations_df.columns[i]])+'</b></td></tr>'
        
        #Add sampling height info:
        html_table = html_table+'<tr><td>sampling height: </td><td><b>'+', '.join(map(str, get_sampl_height_AS_CO2(stations_df.iloc[st].stationId)))+'</b></td></tr>'
        
        #Add html closing tag for table:
        html_table = html_table +'</table>'

        #Get station info in html-format and add it to an iframe:
        iframe = branca.element.IFrame(html=html_table, width=350, height=400)

        #Create dictionary with station lat/lon:
        st_loc_dict = {'lat':stations_df.iloc[st].lat,
                       'lon':stations_df.iloc[st].lon,}

        #Check if current station is selected station:
        if(stations_df.iloc[st].stationId==selected_station):
            
            #Add marker for selected station:
            add_marker(m, st_loc_dict, iframe, 'darkred')
   
        else:
            #Add marker for current station to map:
            add_marker(m, st_loc_dict, iframe, icon_col) 

    #Show map:
    display(m)

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='plotting_funcs'></a>

<br>
<br>

## 5. Plotting functions
This part includes functions that create interactive plots with ICOS CO$_2$ atmospheric data products (Level 1 and Level 2) and STILT results.

In [ ]:
def plot_df_list(df_list, plot_info_dict):
    
    """
    Project:         'ICOS Carbon Portal'
    Author(s):        Anders Dahlner (inspired by Karolina)
    
    Description:      Plots up to 8 columns of a dataframe using 
                      the Bokeh interactive visualization library.
                      Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. A list of, up to three, pandas DataFrames. 
                        
                      2. A dictionary with info on what to plot and what labels to use etc. 

    Output:           Bokeh Figure Object (plot) 
    
    """
    def sub(v:str) -> str:
        # Dictionary for subscript transformations of numbers:
        SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
        return v.replace(tracer,tracer.upper().translate(SUB))
        
    
    n = len(df_list)
    if n not in [1,2,3]:
        print('No data to plot!')
        return
    
    # set default variables
    tracer = ''
    x_axis_label = 'Time (UTC)'
    x_axis_type = 'datetime'
    main_title = ''
    sub_title = ''
    title_align = 'center'
    main_title_font_size = '25px'
    sub_title_font_size = '20px'
    title_offset = 15
    caption_text = '"© ICOS ERIC"'
    caption_font_size = '8pt'
    plot_width = 900
    plot_height = 500
    tools = 'pan,box_zoom,wheel_zoom,undo,reset,save'
    
    if 'tracer' in plot_info_dict.keys():
        tracer = plot_info_dict['tracer']
    if 'x_axis_label' in plot_info_dict.keys(): 
        x_axis_label = plot_info_dict['x_axis_label']
    if 'x_axis_type' in plot_info_dict.keys(): 
        x_axis_type = plot_info_dict['x_axis_type']
    if 'main_title' in plot_info_dict.keys():
        main_title = plot_info_dict['main_title']
    if 'sub_title' in  plot_info_dict.keys():
        sub_title = plot_info_dict['sub_title']
    if 'title_align' in  plot_info_dict.keys():
        title_align = plot_info_dict['title_align']
    if 'main_title_font_size' in  plot_info_dict.keys():
        main_title_font_size = plot_info_dict['main_title_font_size']
    if 'sub_title_font_size' in  plot_info_dict.keys():
        sub_title_font_size = plot_info_dict['sub_title_font_size']
    if 'title_offset' in  plot_info_dict.keys():
        title_offset = plot_info_dict['title_offset']
    if 'caption_text' in  plot_info_dict.keys():
        caption_text = caption_text + '\t'+ plot_info_dict['caption_text']
    if 'caption_font_size' in  plot_info_dict.keys():
        caption_font_size = plot_info_dict['caption_font_size']
    if 'plot_width' in  plot_info_dict.keys():
        plot_width = plot_info_dict['plot_width']
    if 'plot_height' in  plot_info_dict.keys():
        plot_height = plot_info_dict['plot_height']
    if 'tools' in  plot_info_dict.keys():
        tools = plot_info_dict['tools']
        
    x_ax = {}
    left_y_ax = {}
    right_y_ax = {}
    left_y_axis_names = {} 
    right_y_axis_names = {}
    hover_columns = {}
    hover_column_labels = {}
    columns = []
    left_y_axis_label = []
    right_y_axis_label = []
    left_y_column_colors = []
    right_y_column_colors= []
    has_right_y_ax = False
     
    for i in range(n):
        key = f'df_{i}'
        df = df_list[i]
        if key in plot_info_dict.keys():
            data = plot_info_dict[key]
            if 'x_axis_column' in data.keys(): 
                col = data['x_axis_column']
                x_ax[i] = df[col].values
            else:
                x_ax[i] = df.index.values
            if 'left_y_axis_columns' in data.keys():
                columns += data['left_y_axis_columns']
                left_y_ax[i] = []
                for col in data['left_y_axis_columns']:
                    left_y_ax[i].append(df[col].values)
            if 'right_y_axis_columns' in data.keys():
                right_y_ax[i] = []
                has_right_y_ax = True
                for col in data['right_y_axis_columns']:
                    right_y_ax[i].append(df[col].values)
            if ('left_y_axis_columns' not in data.keys()) and \
            ('right_y_axis_columns' not in data.keys()):
                print("Please add \'left_y_axis_columns\' or \'right_y_axis_columns\' into " +\
                      f"the dictionary 'df_{i}' of `plot_info_dict`!")  
                
            if 'left_y_axis_names' in data.keys():
                left_y_axis_names[i] = data['left_y_axis_names'] 
            if 'right_y_axis_names' in data.keys():
                right_y_axis_names[i] = data['right_y_axis_names'] 
            if 'left_y_axis_label' in data.keys() and \
            data['left_y_axis_label'] not in left_y_axis_label:
                left_y_axis_label.append(data['left_y_axis_label'])
            if 'right_y_axis_label' in data.keys():
                right_y_axis_label.append(data['right_y_axis_label'])
            if 'left_y_column_colors' in data.keys():
                left_y_column_colors += data['left_y_column_colors']
            if 'right_y_column_colors' in data.keys():
                right_y_column_colors += data['right_y_column_colors']
            # if 'hover_columns' in data.keys():                        # not implemeted 'hover_column_names'
            #     hover_column_labels = data['hover_columns']
            #     hover_columns[i] = []
            #     for col in data['hover_columns']:
            #         hover_columns[i].append(df[col].values)
                    
        else:
            print(f'The plot_info_dict has no information on dataframe no {i}!')
           # return

        
    len_left_y_ax_cols = sum(len(v) for v in left_y_ax.values())
    if has_right_y_ax:
        len_right_y_ax_cols = sum(len(v) for v in right_y_ax.values())
    else: 
        len_right_y_ax_cols = 0
    
    if len_left_y_ax_cols + len_right_y_ax_cols > 8:
        print('Maximum number of columns to plot is 8.')
        return 

    # If no colors are given we use color addressing color deficiencies
    if len(left_y_column_colors) != len_left_y_ax_cols or \
    len(right_y_column_colors) != len_right_y_ax_cols:
        color_ls = list(availability_plot.get_cb_pallete(len_left_y_ax_cols + len_right_y_ax_cols))
        left_y_column_colors = color_ls[:len_left_y_ax_cols]
        right_y_column_colors = color_ls[len_left_y_ax_cols:]
        
    p = figure(plot_width=plot_width,
               plot_height=plot_height,
               x_axis_label=sub(x_axis_label), 
               y_axis_label=sub('\n'.join(left_y_axis_label)),
               x_axis_type='datetime',
               tools=tools)
    
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 # Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 # Sets the distance of the label from the y-axis in screen units

    if main_title:
        p.title.text = sub(main_title)
        p.title.align = title_align
        p.title.text_font_size = main_title_font_size
    if sub_title:
        p.add_layout(Title(text = sub(sub_title), 
                           align=title_align, 
                           text_font_size = sub_title_font_size),'above')
    
    # Preparing axis(es):
    y1_axis_min = min([min(v) for col in left_y_ax.values() for v in col])
    y1_axis_max = max([max(v) for col in left_y_ax.values() for v in col])
    
    left_y_ax
    
    # Control min/max values for both y-axes to allign their major ticks #####
    #If the difference between min & max values is >120 for both axes:
    if(y1_axis_max-y1_axis_min) > 120: 
        #Round up or down values to nearest 100:
        y1_axis_min = rounddown_100(y1_axis_min - 10)
        y1_axis_max = roundup_100(y1_axis_max + 10)
        y1_limit = 50.0
    else:
        #Round up or down values to nearest 20:
        y1_axis_min = rounddown_20(y1_axis_min - 10)
        y1_axis_max = roundup_20(y1_axis_max + 10)
        y1_limit = 20.0
    
    if has_right_y_ax:
        y2_axis_min = min([min(v) for col in right_y_ax.values() for v in col])
        y2_axis_max = max([max(v) for col in right_y_ax.values() for v in col])

        if (y2_axis_max-y2_axis_min) > 120:
            #Round up or down values to nearest 100:
            y2_axis_min = rounddown_20(y2_axis_min - 10)
            y2_axis_max = roundup_20(y2_axis_max + 10)
            y2_limit = 50.0
        else:
            #Round up or down values to nearest 20:
            y2_axis_min = rounddown_20(y2_axis_min - 10)
            y2_axis_max = roundup_20(y2_axis_max + 10)
            y2_limit = 20.0

        #Set primary and secondary y-axis range, so that they are alligned:
        p.y_range, p.extra_y_ranges = set_yranges_2y(y1_axis_min, y1_axis_max,
                                                     y2_axis_min, y2_axis_max,
                                                     y1_limit, y2_limit, 'Yaxis2')
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=y1_limit)
        ticker2 = SingleIntervalTicker(interval=y2_limit)
    
    else:
        y_range = Range1d(start = y1_axis_min, end = y1_axis_max)
        
        #Set y-axes ticker interval:
        ticker1 = SingleIntervalTicker(interval=y1_limit)

    # Create glyphs an legend:
    k = 0
    g = {}
    left_legend_ls = []
    right_legend_ls = []
    for i in range(n):
        for j in range(len(left_y_ax[i])):
            g[k] = [] 
            color = left_y_column_colors.pop(0)
            g[k].append(p.circle(x_ax[i], left_y_ax[i][j], radius=.02, color=color))
            g[k].append(p.line(x_ax[i], left_y_ax[i][j], line_width=1.5, color=color, name=left_y_axis_names[i][j]))
            left_legend_ls.append((g[k][1].name, [g[k][0], g[k][1]]))
            k+=1
        if i in right_y_ax.keys():
            for j in range(len(right_y_ax[i])):
                g[k] = [] 
                color = right_y_column_colors.pop(0)            
                g[k].append(p.circle(x_ax[i], right_y_ax[i][j], radius=.02, color=color))
                g[k].append(p.line(x_ax[i], right_y_ax[i][j], line_width=1.5, color=color, name=right_y_axis_names[i][j], y_range_name='Yaxis2'))
                right_legend_ls.append((g[k][1].name, [g[k][0], g[k][1]]))
                k+=1

    # HoverTool best used after with one graph at a time, use the legend to unselect glyphs
    # the 
    p.add_tools(HoverTool(tooltips=[('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
                                    (sub(tracer),'@y{0.f}'),
                                    ('Category', '$name')],   # name: is from the glyphs
                          formatters={'@x': 'datetime'},      # use 'datetime' formatter for 'date' field
                          mode='vline'))                      # vline: display a tooltip whenever the cursor is vertically in line with a glyph   
    
    #Create legend:
    legend = Legend(items=left_legend_ls + right_legend_ls, location = 'left')
    legend.border_line_width = 0
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 20 #sets the distance between legend entries
    # left_legend.label_standoff = 5
    legend.label_width = 20
    p.add_layout(legend, 'below')
    
    #Set primary y-axis ticker:
    p.yaxis.ticker = ticker1
    
    if has_right_y_ax: 
        #Create 2nd y-axis: 
        bg_yaxis = LinearAxis(y_range_name='Yaxis2',
                              axis_label=sub('\n'.join(right_y_axis_label)),
                              axis_label_text_font_style = 'normal',
                              ticker=ticker2,
                              axis_label_standoff = 15)
        p.add_layout(bg_yaxis, 'right')
    

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    

    #Define output location:
    output_notebook()

    #Show plot
    show(p)
        


In [ ]:
def plot_df(stilt_station_id):
    # This function is not used at this moment, but could be useful. 
    
    # Below are some utility functions that parse data from dobj.meta.
    def dobj_column_unit(dobj, column_label):
        # Returns: The unit of the data of the column, if applicable 
        df_var = dobj.variables
        return df_var.loc[df_var.name == column_label].unit.iloc[0] or ''

    def dobj_column_type(dobj, column_label):
        # Returns: The a type for the column
        df_var = dobj.variables
        return df_var.loc[df_var.name == column_label].type.iloc[0] or ''


    def dobj_sampling_height(dobj):
        # Returns: A string for the sampling height of the measured data, if applicable 
        try:
            if dobj.meta['specificInfo']['acquisition']['samplingHeight'] > 0:
                return '(' + str(dobj.meta['specificInfo']['acquisition']['samplingHeight']) + ' m.a.g.l.)' 
            else:
                return ''
        except: 
            return ''

    def dobj_station_name(dobj):
        # Returns: The station name where the data were sampled
        try:
            return str(dobj.station['org']['name'])
        except:
            return ''

    def dobj_product(dobj):
        # Returns: product if any
        try:
            return str(dobj.meta['specification']['self']['label'])
        except:
            return ''
    
    def plot_2(df1, df2,  station_info_dict, tracer_info_dict, level=2, color='#0F0C08'):

        from bokeh.plotting import figure, show
        from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
        from datetime import datetime

        #Dictionary for subscript transformations of numbers:
        SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
        SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

        #Define Datasets:
        x = pd.to_datetime(df1['TIMESTAMP'], unit='ms')
        y = df1['co2']  
        z = df1["NbPoints"].values
        w = df1["Stdev"].values
        o = df1["Flag"].values
        
        xx = pd.to_datetime(df2['TIMESTAMP'], unit='ms')
        yy = df2['co2'] 
        zz = df2["NbPoints"].values
        ww = df2["Stdev"].values
        oo = df2["Flag"].values

        #Create a ColumnDataSource object:
        source = ColumnDataSource(  data = {'x':x, 'y':y, 'z':z, 'w':w, 'o':o}  )
        source2 = ColumnDataSource( data = {'x':xx, 'y':yy, 'z':zz, 'w':ww, 'o':oo} )

        #Create a figure object:
        p = figure(plot_width=900,
                   plot_height=400,
                   x_axis_label='Time (UTC)', 
                   y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+ ' (' +
                   tracer_info_dict['tracer_unit'].translate(SUP) + ')',
                   x_axis_type='datetime',
                   title = tracer_info_dict['tracer_info'].translate(SUB)+'    '+
                   station_info_dict['station_name'] +', '+
                   station_info_dict['station_country'] +', '+
                   station_info_dict['station_sampling_height'] +' m.a.g.l.',
                   tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

        #Create glyphs:
        g0 = p.circle('x','y', source=source, radius=.02, color=color)

        #If data is level-2 data:
        #if(level==2):
        g1 = p.line('x','y', source=source, line_width=1, color=color,
                    name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))

        #If data is level-1 data:
        #else:
        g1 = p.line('x','y', source=source2, line_width=2, color=color,  
                    name='xxxxxx' + tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),
                    line_dash='dotted')


        #Add tooltip on hover:
        p.add_tools(HoverTool(tooltips=[
            ('Station Code',station_info_dict['station_code']),
            ('Latitude',station_info_dict['station_lat']),
            ('Longitude',station_info_dict['station_lon']),
            ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
            (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
            ('St dev', '@w{0.f}'),
            ('NbPoints', '@z'),
            ('Flag', '@o')
            ],
            formatters={
                '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
                },
            # display a tooltip whenever the cursor is vertically in line with a glyph
            mode='vline'
            ))    


        #Set title attributes:
        p.title.align = 'center'
        p.title.text_font_size = '13pt'
        p.title.offset = 15

        #Set label font style:
        p.xaxis.axis_label_text_font_style = 'normal'
        p.yaxis.axis_label_text_font_style = 'normal'
        p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
        p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

        #Set the copyright label position:
        label_opts = dict(x=0, y=10,
                          x_units='screen', y_units='screen')

        #Create a label object and format it:
        caption1 = Label(text="© ICOS ERIC", **label_opts)
        caption1.text_font_size = '8pt'

        #Deactivate hover-tool, which is by default active:
        p.toolbar.active_inspect = None

        #Add label to plot:
        p.add_layout(caption1, 'below')

        #return plot:
        return p

    df= availability_df.loc[(availability_df.stiltStationId == stilt_station_id) & (availability_df.datalevel=='2')]
    if len(df)>0:
        station_code = df.stationId.values[0]
        pid1 = df.dobj.values[0]
        dobj1 = Dobj(pid1)
        lev2_df = dobj1.data
    else:
        print('No ICOS level 2 data available.')

    df= availability_df.loc[(availability_df.stiltStationId == stilt_station_id) & (availability_df.datalevel=='1')]
    if len(df)>0:
        pid2 = df.dobj.values[0]
        dobj2= Dobj(pid2)
        product2 = dobj_product(dobj2)
        lev1_df = dobj2.data
    else:
        print('No ICOS level 1 data available.')
        df2 = pd.DataFrame()

    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = dobj_column_type(dobj1,'co2')
    tracer_info_dict['tracer_unit'] = dobj_column_unit(dobj1,'co2')
    
    station_sampl_height = dobj_sampling_height(dobj1)
    
    #Loop through station obj list:
    for st in atm_stations:
        
        if st.stationId==station_code:
            
            #Get station info:
            station_info_dict['station_name'] = st.name
            station_info_dict['station_code'] = station_code
            station_info_dict['station_sampling_height'] = station_sampl_height
            station_info_dict['station_country_code'] = st.country
            station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(st.country)
            station_info_dict['station_lat'] = str(st.lat)
            station_info_dict['station_lon'] = str(st.lon)
        
    
    #Call plotting function:
    p = plot_2(lev2_df,lev1_df, station_info_dict, tracer_info_dict,)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p)
   


<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='footprint_funcs'></a>

<br>
<br>

## 6. STILT Footprint functions
This part includes functions that read STILT footprints and visualize them in static maps.

In [ ]:
def read_aggreg_footprints(station, date_range, stilt_icos_availability_df, pathFP, timeselect='all'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Nov 09 14:00:00 2018
    Last Changed:     Mon May 13 12:00:00 2018
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to read and aggregate footprints for given time range.
                      
    Input parameters: 1. STILT Station ID
                         (var_name: "station", var_type: String)
                      2. Set of Datetime Objects
                         (var_name: "date_range", var_type: Pandas Dataframe)
                      3. Dataframe with info about ICOS Atmospheric Stations with STILT results
                         (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)
                      4. Time Selection, available options: ('all', 'daytime', 'nighttime')
                         (var_name: "timeselect", var_type: String)

    Output:           1. Total number of Footprints available for the given station & date range
                         (var_name: "nfp", var_type: Integer)
                      2. Aggregated footprint
                         (var_name: "fp", var_type: Masked Array of Floats)
                      3. Longitudes
                         (var_name: "lon", var_type: Masked Array of Floats)
                      4. Latitudes
                         (var_name: "lat", var_type: Masked Array of Floats)
                      5. Title of plot
                         (var_name: "title", var_type: String)
    
    """
    
    #Create and initialize variables:
    fp=[]        #List to store footprints
    nfp=0        #Store total number of footprints
    first = True #Control variable -> cotrols 1st loop iteration
    
    # loop over all dates and read netcdf files
    for dd in date_range:
        filename=(pathFP+
                  stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==station].values[0]+'/'+
                  str(dd.year)+'/'+str(dd.month).zfill(2)+'/'+
                  str(dd.year)+'x'+str(dd.month).zfill(2)+'x'+
                  str(dd.day).zfill(2)+'x'+str(dd.hour).zfill(2)+'/foot')
        
        #If path to footprint exists, read netcdf-file:
        if os.path.isfile(filename):
            f_fp = cdf.Dataset(filename)
            
            #If this is the 1st iteration,
            #get footprint as well as latitudes & longitudes:
            if (first):
                fp=f_fp.variables['foot'][:,:,:]
                lon=f_fp.variables['lon'][:]
                lat=f_fp.variables['lat'][:]
                
                #Set variable controling check of 1st loop iteration to False:
                first = False
                
            #If this is not the first iteration of the loop:    
            else:
                
                #Read footprint:
                fp=fp+f_fp.variables['foot'][:,:,:]
            
            #Close file:
            f_fp.close()
            
            #Increase the counter of total num of footprints:
            nfp+=1        
    
    
    #If the total number of footprints is greater than zero:
    if nfp > 0:
        
        #Aggregate footprints:
        fp=fp/nfp
    

    #If no footprints are found:
    if(nfp==0):
        
        #Set output values:
        nfp = 0
        fp = None
        lon = None
        lat = None 
        title = ""
    
    
    #If there are footprints:
    else:
        #Create variable to store the footprint-map title:
        title = (date_range.min().strftime('%Y-%m-%d')+' - '+date_range.max().strftime('%Y-%m-%d')+'\n'+
                 'time selection: '+timeselect)

    #Return output:
    return nfp, fp, lon, lat, title

In [ ]:
def lonlat_2_ixjy(slon,slat,mlon,mlat):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon May 13 15:00:00 2019
    Last Changed:     Mon May 13 15:00:00 2019
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to convert station longitude and latitude (slat, slon) to indices
                      of STILT model grid (ix,jy)
                      
    Input parameters: 1. ICOS Station Longitude
                         (var_name: 'slon', var_type: Float)
                      2. ICOS Station Latitude
                         (var_name: "slat", var_type: Float)
                      3. Longitudes of the STILT Model Grid
                         (var_name: "mlon", var_type: Array of Floats)
                      4. Latitudes of the STILT Model Grid
                         (var_name: "mlat", var_type: Array of Floats)
                      

    Output:           Integer Variables (2)
    
    """
    
    #Get the STILT Model Grid-indices for the ICOS Station lat/lon:
    ix = (np.abs(mlon-slon)).argmin()
    jy = (np.abs(mlat-slat)).argmin()
    
    #Return STILT Model Grid indices:
    return ix,jy

In [ ]:
# function to plot maps (show station location if station is provided and zoom in second plot if zoom is provided)
def plot_fp_maps(field, lon, lat, station_info_df,
                 title='', label='', unit='', linlog='linear', station_id='',
                 vmin=None, vmax=None, colors='GnBu',midpoint=False):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon May 13 15:00:00 2019
    Last Changed:     Mon May 13 15:00:00 2019
    Version:          1.0.0
    Author(s):        Ute, Karolina
    
    Description:      Function to plot footprint maps, depicting the station location. This function returns two
                      matplotlib plots with the footprints for a given ICOS Atmosphere Station for a given time
                      period. The first plot presents the STILT footprint over an area that covers the entire
                      European continent. The second plot presents the footprint over a smaller area around the
                      station.
                      
    Input parameters:  1. STILT Footprints for a specific ICOS Station
                          (var_name: 'field', var_type: Array)
                       2. Longitudes of the STILT Model Grid
                          (var_name: "lon", var_type: Array of Floats)
                       3. Latitudes of the STILT Model Grid
                          (var_name: "lat", var_type: Array of Floats)
                       4. Station info Dataframe
                          (var_name: "station_info_df", var_type: Pandas DataFrame)
                       5. Footprint Map Title
                          (var_name: "title", var_type: String)
                       6. Footprint Map Label
                          (var_name: "label", var_type: String)
                       7. Footprint Map Units
                          (var_name: "unit", var_type: String)
                       8. Colorscale Color Assignment Method
                          (var_name: "linlog", var_type: String)
                       9. String with ICOS station id
                          (var_name: "station_id", var_type: String)
                      10. Colorscale Lower Limit Footprint Value
                          (var_name: "vmin", var_type: Float)
                      11. Colorscale Upper Limit Footprint Value
                          (var_name: "vmax", var_type: Float)
                      12. Colorscale Name
                          (var_name: "colors", var_type: String)
                      13. Midpoint for diverging colorscales
                          (var_name: "midpoint", var_type: Float or Boolean)
                      

    Output:           Matplotlib Footprint Maps (if available)
    
    """
    
    import matplotlib.pyplot as plt

    
    #Check the dimensions of the Footprint array:
    if np.shape(field)[0] > 1:
        print ('More than one field: ',np.shape(field)[0],' Only the first will be plotted!!!')
    
    #Create figure and set figure size:
    fig = plt.figure(figsize=(15,8))

    # set up a map
    ax = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
    img_extent = (lon.min(), lon.max(), lat.min(), lat.max())
    #ax.set_extent([-15,35,33,72],crs=ccrs.PlateCarree())
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()],crs=ccrs.PlateCarree())
    #ax.gridlines(draw_labels=True)

    # Create a feature for Countries at 1:50m from Natural Earth
    countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    #ax.coastlines(resolution='50m', color='white', linewidth=0.3

    #cmap = p.get_cmap('Blues')
    #cmap = p.get_cmap('GnBu')
    cmap = plt.get_cmap(colors)
    if linlog == 'linear':
        print(' ')
        im=ax.imshow(field[0,:,:], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='neither')
        cbar.set_label(label+'  '+unit)
    
    else:
        im=ax.imshow(np.log10(field)[0,:,:], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.05, fraction=0.055,extend='neither')
        cbar.set_label(label+'  log$_{10}$ '+unit)
    plt.title(title)
    ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(field[0,:,:]), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(field[0,:,:]), horizontalalignment='right',transform=ax.transAxes)
    
   
    if station_id != '':
        #show station location if station is provided
        ax.plot(float(station_info_df.lon.loc[station_info_df['stationId']==station_id].values[0]),
                float(station_info_df.lat.loc[station_info_df['stationId']==station_id].values[0]),
                'm+',markersize=8,transform=ccrs.PlateCarree())
    #ax.plot(10.0,53.0,'m+',markersize=8,transform=ccrs.PlateCarree())
 


    #grid cell index of station 
    ix,jy = lonlat_2_ixjy(float(station_info_df.lon.loc[station_info_df['stationId']==station_id].values[0]),
                          float(station_info_df.lat.loc[station_info_df['stationId']==station_id].values[0]),
                          lon,lat)

    # define zoom area 
    i1 = np.max([ix-35,0])
    i2 = np.min([ix+35,400])
    j1 = np.max([jy-42,0])
    j2 = np.min([jy+42,480])

    # set up a map
    ax = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
    img_extent = (lon[i1:i2].min(), lon[i1:i2].max(), lat[j1:j2].min(), lat[j1:j2].max())
    #ax.set_extent([-15,35,33,72],crs=ccrs.PlateCarree())
    ax.set_extent([lon[i1:i2].min(), lon[i1:i2].max(), lat[j1:j2].min(), lat[j1:j2].max()],crs=ccrs.PlateCarree())
    #ax.gridlines(draw_labels=True)

    # Create a feature for Countries at 1:50m from Natural Earth
    countries = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')
    ax.add_feature(countries, edgecolor='black', linewidth=0.3)
    #ax.coastlines(resolution='50m', color='white', linewidth=0.3

    if linlog == 'linear':
        print(' ')
        im=ax.imshow(field[0,j1:j2,i1:i2], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.03,fraction=0.055,extend='neither')
        cbar.set_label(label+'  '+unit)
        
    else:
        
        im=ax.imshow(np.log10(field)[0,j1:j2,i1:i2], origin='lower', extent=img_extent,vmin=vmin,vmax=vmax,cmap=cmap)
        cbar=plt.colorbar(im,orientation='horizontal',pad=0.05,fraction=0.055,extend='neither')
        cbar.set_label(label+'  log$_{10}$ '+unit)
    
    plt.title(title)
    ax.text(0.01, -0.27, 'min: %.5f' % np.nanmin(field[0,j1:j2,i1:i2]), horizontalalignment='left',transform=ax.transAxes)
    ax.text(0.99, -0.27, 'max: %.5f' % np.nanmax(field[0,j1:j2,i1:i2]), horizontalalignment='right',transform=ax.transAxes)
        
    if station != '':
        
        #show station location if station is provided
        ax.plot(float(station_info_df.lon.loc[station_info_df['stationId']==station_id].values[0]),
                float(station_info_df.lat.loc[station_info_df['stationId']==station_id].values[0]),
                'm+',markersize=8,transform=ccrs.PlateCarree())
        
    #Show plots:
    plt.show()
    plt.close()
   

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='widget_func'></a>

<br>
<br>

## 7. Widget functions
This part includes functions that create widget forms. Widget are interactive elements like dropdown lists, checkboxes, etc. in Python.

In [ ]:
#Create function for ICOS AS map widget form:
def map_wdgt_form_icos_stilt():
    
    #Import modules:
    from ipywidgets import VBox, Button, Dropdown, Output, Label
    from IPython.display import clear_output

    st_info_df = get_position_table()
    
    # Create dropdown list for ICOS stations:
    stations_wdgt =  Dropdown(options = sorted([(st_info_df['name'].iloc[i],
                                                 st_info_df['stationId'].iloc[i])
                                                for i in range(len(st_info_df))]), 
                              description='Station')
    
    # Create dropdown list basemap options:
    basemap_wdgt = Dropdown(options = ['Imagery', 'OpenStreetMap'], description='Basemap')
    
    # Create button widget (execution):
    button_exe = Button(description='Update map',
                        disabled=False,
                        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip='Press the button to update map',
                        icon='check')
    
    
    # Format widgets:
    stations_wdgt.layout.width = '379px'
    stations_wdgt.style.description_width = 'initial'
    stations_wdgt.layout.margin = '2px 2px 2px 238px'
    basemap_wdgt.layout.width = '394px'
    basemap_wdgt.style.description_width = 'initial'
    basemap_wdgt.layout.margin = '2px 2px 2px 224px'
    button_exe.style.button_color = '#3973ac'
    button_exe.layout.width = '250px'
    button_exe.layout.margin = '50px 100px 40px 330px'
    
    # Create form object:
    form_out = Output()

    # Create output object:
    plot_out = Output()
    
    # Function that executes on button_click (calculate score):
    def on_exe_bttn_clicked(button_c):
    
        #Open output object:
        with plot_out:

            #Delete previous output:
            clear_output()

            #Show new map:
            plotmap_stilt(st_info_df, stations_wdgt.value, basemap_wdgt.value, d_icon='cloud', icon_col='orange')
            
    #Call function on button_click-event (calculate score):
    button_exe.on_click(on_exe_bttn_clicked)

    #Open output obj:a
    with form_out:

        #Clean previous values:
        clear_output()

        #Show plot:
        display(VBox([stations_wdgt, basemap_wdgt, button_exe, plot_out]))

    #Display form:
    display(form_out)

In [ ]:
def station_drop_list():
    # Returns a list of stations used for dropdowns of the timesseies and footprint plots,
    # each element of the list is a tuple with a name and a dictionary, of the form: 
    # ('STILT-station name', 'stiltStationId': str, 'stiltAlt': int, 'stationId': str, 'samplingheight': str} )
    # e.g.
    # ('Gartow 60m', {'stiltStationId': 'GAT060', 'stiltAlt': 60, 'stationId': 'GAT', 'samplingheight': '60.0'})
    #
    # The list is sorted with respect to station names and altitude 
    # e.g 'Gartow 60m', 'GAT060' is sorted as 'Gartow060', 
    # while 'Monte Cimone 760m', 'CMN760' is sorted as 'Monte760'
    # [..., ('Gartow 60m', {'stiltStationId': 'GAT060', 'stiltAlt': 60, 'stationId': 'GAT', 'samplingheight': '60.0'}), 
    #     , ('Gartow 132m', {'stiltStationId': 'GAT132', 'stiltAlt': 132, 'stationId': 'GAT', 'samplingheight': '132.0'}),
    #  ..., ('Lutjewad 60m', {'stiltStationId': 'LUT', ...), ('Monte Cimone 760m', {'stiltStationId': 'CMN760',...),...]

    avail_df = get_timeseries_availability_stilt_icos()
    column_list = ['stiltStationId', 'stiltAlt', 'stationId', 'samplingheight']
    df = avail_df[['stiltName'] + column_list].drop_duplicates()
        
    # Sorting the df:
    df = df.loc[(df.stiltName.apply(lambda x: x.split(' ')[0]) + df.stiltStationId.apply(lambda x: x[3:])).sort_values().index]

    drop_list = []
    for x in range(len(df)):
        drop_list.append((df.iloc[x,0],dict(zip(column_list, list(df.iloc[x,1:])))))

    return drop_list



In [ ]:
def create_widgets_stilt_icos_plots():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 14:00:00 2019
    Last Changed:     Wed Oct 12 14:00:00 2022
    Version:          1.2.0
    Author(s):        Karolina Pantazatou
                      Anders Dahlner
    
    Description:      Function that creates a set of widgets; a station dropdown list, a year dropdown list,
                      a citation checkbox and a button, populates the dropdown lists with values, captures the 
                      user's input and calls a function to update the contents of the plot. If the citation checkbox
                      is checked, a string including the citation text for all data layers included in the plot
                      will be displayed.
    

    Output:           Plot and/or Citation and/or Warning Message
    
    """

    def year_drop_list(current_stationId):
        # A sorted list of years where there are 
        # STILT data or ICOS Level 1 or Level 2 observations
        return list(set(year_df['year'].loc[year_df.stiltStationId == current_stationId]))

    def station_drop_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            year_dropdown.options = year_drop_list(station_dropdown.value['stiltStationId'])
    
    # We will use the table year_df below as a lookup table for availability
    time_df_list = list_of_availability_dataframes()[0:3]
    time_df_list[0]['type'] = 'ICOS Level 1'
    time_df_list[1]['type'] = 'ICOS Level 2'
    time_df_list[2]['type'] = 'STILT'
    
    year_df = pd.concat(time_df_list)
    year_df = year_df[['stiltStationId','year','type']].drop_duplicates().sort_values(['stiltStationId','year','type'])
    
    # Dropdown of ICOS Atmospheric stations for which STILT model results 
    # or ICOS Level 1 or Level 2 data are available:
    station_dropdown =  Dropdown(options = station_drop_list(),
                        description='Station')
    
    year_dropdown = Dropdown(options = year_drop_list(station_dropdown.value['stiltStationId']))
    
    station_dropdown.observe(station_drop_change)
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(station_value, year_value, citation_value):
        from icoscp.stilt import stiltstation
        
        station_name = [x[0] for x in station_dropdown.options if x[1] == station_value].pop()
        stilt_station_id = station_value['stiltStationId']
        stilt_alt = str(station_value['stiltAlt'])
        
        available_types = set(year_df.loc[(year_df.stiltStationId == stilt_station_id) & 
                                          (year_df.year == year_value)].type)
        #Define start date and end date:
        start_date= date(year_value,1,1)
        end_date= date(year_value,12,31)

        plots_for_dates(stilt_station_id, station_name, stilt_alt, start_date, end_date, citation_value)
            

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 station_value=station_dropdown,
                                 year_value=year_dropdown,
                                 citation_value=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

In [ ]:
def plots_for_dates(stilt_station_id, station_name, stilt_alt, start_date, end_date, citation_value):
    # This is used by: 
    #  - create_widgets_stilt_icos_plots()
    #  - create_widgets_stilt_icos_fp()
    
    availability_df = get_timeseries_availability_stilt_icos()
    
    no_data_avail_list = []
    citation_list = []
    plot_dicts_ls = []
    dataframes_to_plot_ls = []

    # 1. Fetch STILT-data and meta-data
    stilt_stn_obj = stiltstation.get(id=stilt_station_id)
    stilt_df = stilt_stn_obj.get_ts(start_date,end_date) 
    if stilt_df.empty:
        no_data_avail_list.append('STILT data') 
    else:           
        plot_dicts_ls.append(stilt_plot_dict(station_name, stilt_alt))
        dataframes_to_plot_ls.append(stilt_df)

        if(citation_value):
            citation_list.append("<sub><b>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"<b></sub>")


    # We gather ICOS data in three steps in order to fix a nice title.
    # 2.1 level 1 data
    dobj_ls = []
    df = availability_df.loc[(availability_df.stiltStationId == stilt_station_id) & (availability_df.datalevel=='1')]
    if len(df)>0:
        station_code = df.stationId.values[0]
        pid = df.dobj.values[0]
        dobj1 = Dobj(pid)
        icos_l1_df = dobj1.data
        icos_l1_df.set_index('TIMESTAMP', drop=True, inplace=True)
        icos_l1_df.sort_index(inplace=True)
        icos_l1_df = icos_l1_df[start_date:end_date]
        if icos_l1_df.empty:
            no_data_avail_list.append('ICOS Level 1 data')
            dobj1 = None
        else:
            dobj_ls.append((dobj1,1))
            dataframes_to_plot_ls.append(icos_l1_df)
            if(citation_value):
                citation_list.append("<sub><b>" + dobj1.citation + "<b></sub>")

    # 2.2 level 2 data
    df = availability_df.loc[(availability_df.stiltStationId == stilt_station_id) & (availability_df.datalevel=='2')]
    if len(df)>0:
        station_code = df.stationId.values[0]
        pid = df.dobj.values[0]
        dobj2 = Dobj(pid)
        icos_l2_df = dobj2.data
        icos_l2_df.set_index('TIMESTAMP', drop=True, inplace=True)
        icos_l2_df.sort_index(inplace=True)
        icos_l2_df = icos_l2_df[start_date:end_date]
        if icos_l2_df.empty:
            no_data_avail_list.append('ICOS Level 2 data')
            dobj2 = None
        else:
            dobj_ls.append((dobj2,2))
            dataframes_to_plot_ls.append(icos_l2_df)
            if(citation_value):
                citation_list.append("<sub><b>" + dobj2.citation + "<b></sub>")
    # 2.3. Level 1 and/or level 2 meta-data
    if dobj_ls:
        icos_plot_dicts = icos_plot_dict(dobj_ls)
        for k in icos_plot_dicts.keys():
            plot_dicts_ls.append(icos_plot_dicts[k]) 

    # 3. Prepare the meta-data
    if plot_dicts_ls:
        plot_meta_dict = plot_dict()
        k = 0 
        while plot_dicts_ls:
            key = f'df_{k}' 
            plot_meta_dict[key] = plot_dicts_ls.pop(0)
            k+=1
        # set titles
        if k == 1:
            key = f'df_{0}'
            plot_meta_dict['main_title'] = plot_meta_dict[key]['single_plot_main_title']
            plot_meta_dict['sub_title'] = plot_meta_dict[key]['single_plot_sub_title']
        else:
            j = 0
            title_ls = []
            while j < k:
                key = f'df_{j}'
                title_ls.append(plot_meta_dict[key]['title'])
                j+=1
            plot_meta_dict['sub_title'] ='\n'.join(title_ls)
        
        # 4. Plot the data
        plot_df_list(dataframes_to_plot_ls,plot_meta_dict)
    
    # 5. Print availability info 
    if no_data_avail_list:
        print_string = ''
        for x in range(len(no_data_avail_list)):
            if x == 0:
                print_string = f"<sub><b>No {no_data_avail_list[0]}"
            else:
                print_string += f" or {no_data_avail_list[x]}"
        print_string += " available.<b></sub><br>"
        printmd(print_string)

    # 6. Cite.
    if(citation_list):
        for x in citation_list:
            printmd(x)
    
    return


def plot_dict():
    return {'tracer': 'CO2',
            'x_axis_label': 'Time (UTC)',
            'x_axis_type': 'datetime',
            'title_align': 'center',
            'main_title_font_size': '25px',
            'sub_title_font_size': '20px',
            'title_offset': 15,
            'caption_text': '"© ICOS ERIC"',
            'caption_font_size': '8pt',
            'plot_width': 900,
            'plot_height': 500,
            'tools': 'pan,box_zoom,wheel_zoom,undo,reset,save'}
    
                      
def stilt_plot_dict(stilt_stn_id, stilt_alt):
    
    # Get a list of five colors addressing color deficiencies
    from tools2.visualization.availability import availability_plot
    five_colors = availability_plot.get_cb_pallete(5)
    tracer = 'CO2'
    
    return {'single_plot_main_title': f'STILT Model Output of {tracer} with main components.',
            'single_plot_sub_title': f'Station: {stilt_stn_id}. Calculation height: {stilt_alt} m.a.g.l.',   
            'title': f'STILT Model for station: {stilt_stn_id}. {tracer} calculation height: {stilt_alt} m.a.g.l.', 
            'left_y_axis_columns': ['co2.stilt','co2.background'],
            'right_y_axis_columns': ['co2.bio','co2.fuel','co2.cement'],
            'left_y_axis_names': ['STILT','Background'],                             # for legend
            'right_y_axis_names': ['Bio','Fuel','Cement'],                           # for legend
            'left_y_axis_label': f'CO2 (ppm): STILT & Background component',
            'right_y_axis_label': f'STILT-CO2-components (ppm): Bio, Fuel & Cement',
            'left_y_column_colors': [five_colors[0],five_colors[1]],
            'right_y_column_colors': [five_colors[3],five_colors[2],five_colors[4]], # here we use green for 'bio'
            'hover_columns': ['co2.stilt','co2.background','co2.bio','co2.fuel','co2.cement'],
            'hover_column_names': ['Time (UTC)','Stilt','Background','Bio','Fuel','Cement']}
      
def icos_plot_dict(dobj_ls):
    
    # Below are some utility functions that parse data from dobj.meta.
    def dobj_column_unit(dobj, column_label):
        # Returns: The unit of the data of the column, if applicable 
        df_var = dobj.variables
        return df_var.loc[df_var.name == column_label.lower()].unit.iloc[0] or ''

  
    def dobj_sampling_height(dobj):
        # Returns: A string for the sampling height of the measured data, if applicable 
        try:
            if dobj.meta['specificInfo']['acquisition']['samplingHeight'] > 0:
                return str(dobj.meta['specificInfo']['acquisition']['samplingHeight']) + ' m.a.g.l.' 
            else:
                return ''
        except: 
            return ''

    def dobj_station_name(dobj):
        # Returns: The station name where the data were sampled
        try:
            return str(dobj.station['org']['name'])
        except:
            return ''

    def dobj_product(dobj):
        # Returns: product if any
        try:
            return str(dobj.meta['specification']['self']['label'])
        except:
            return ''
    
    tracer = 'CO2'
    
    d = dict()

    for dobj, level in dobj_ls:

        if level == 1:
            color = 'gray'
        else: 
            color = 'black'
       
        d[level] = {'single_plot_main_title': f'ICOS Station {dobj_station_name(dobj)} {tracer}-measurements.',
            'single_plot_sub_title': f'{dobj_product(dobj)} (Level {level}), sampling-height: {dobj_sampling_height(dobj)}.',
            'title': f'ICOS Station {dobj_station_name(dobj)}, {dobj_product(dobj)} (Level {level}), height {dobj_sampling_height(dobj)}',
            'left_y_axis_columns': ['co2'],
            'left_y_axis_names': [f'ICOS Level {level} CO2'],              # for legend
            'left_y_axis_label': f'CO2 ({dobj_column_unit(dobj,tracer)})',
            'left_y_column_colors': [color],
            'hover_columns': ['Flag', 'NbPoints', 'Stdev', 'co2'],
            'hover_column_names': ['Time (UTC)','CO2','St dev','NbPoints','Flag']}
    if len(dobj_ls)==2:
        dobj1 = dobj_ls[0][0]
        level1 = dobj_ls[0][1]
        dobj2 = dobj_ls[1][0]
        level2 = dobj_ls[1][1]
        d[level1]['title'] = f'ICOS Station {dobj_station_name(dobj1)}. {dobj_product(dobj1)} (Level {level1}) and'
        d[level2]['title'] = f'{dobj_product(dobj2)} (Level {level2}), sampling-height {dobj_sampling_height(dobj1)}'
    
    return d
    


In [ ]:
def create_widgets_stilt_icos_fp():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Sat May 11 14:00:00 2019
    Last Changed:     Sat May 11 14:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station dropdown list, two datepickers, 
                      two checkboxes and a button, populates the dropdown lists with values, captures the 
                      user's input and calls a function to update the contents of the plot. The function
                      also outputs a set of Footprint maps and data citation strings, if the corresponding
                      checkboxes are checked.
                      
    Input parameters: Dataframe with info about ICOS Atmospheric Stations with STILT results
                      (var_name: 'stilt_icos_availability_df', var_type: Pandas Dataframe)

    Output:           Plot with Map or Warning Message
    
    """
    def valid_dates(start_date, end_date):
        #Check if either or both of the selected start_date or end_date are equal to NULL:
        if((start_date==None)|(end_date==None)):
            print('\033[0;31;1m No date(s) selected!\n\n')
            return False
        else:
            try: 
                diff = end_date - start_date
                if(diff.days<0):
                    print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')
                    return False
                else:
                    return True
            except:
                pass
                return False
    
    # path to foot print data
    path_stilt = CPC.STILTFP
    
    stilt_icos_availability_df = get_timeseries_availability_stilt_icos()
    station_info_df = get_position_table()
    
    #Get a list of labels of ICOS Atmospheric stations for which STILT model results are available:
    init_station_ls =  station_drop_list()  
    
    #Remove test-result for SMEAR:
    stations_updated = [x for x in init_station_ls if x!=('SMEAR II-ICOS Hyytiälä (alt. 127.0)', ['SMR127', '40.0', 'SMR', '125.0'])]

    #Create widgets:
    station_dropdown =  Dropdown(options = stations_updated,
                        description='Station')


    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(station_value, start_date, end_date, Footprint, Citation):
        
        if valid_dates(start_date, end_date):
            
            stilt_stn_id = station_value['stiltStationId']
            stilt_alt = str(station_value['stiltAlt'])
            icos_stn_id = station_value['stationId']
            icos_stn_sh = station_value['samplingheight']
            station_name = [x[0] for x in station_dropdown.options if x[1] == station_value].pop()
            
            #Create a pandas dataframe containing one column of datetime objects with 3-hour intervals:
            date_range = pd.date_range(start_date, end_date + dt.timedelta(hours=24), freq='3H')
            
            plots_for_dates(stilt_stn_id, station_name, stilt_alt, start_date, end_date, Citation)
            
            #Get STILT-data:
            stilt_stn_obj = stiltstation.get(id=stilt_stn_id)
            stilt_df = stilt_stn_obj.get_ts(start_date,end_date) 
            
            if(Footprint):

                #Get footprint data and plot map:
                stilt_nfp, stilt_fp, stilt_lon, stilt_lat, stilt_title = read_aggreg_footprints(stilt_stn_id,
                                                                                                date_range,
                                                                                                stilt_icos_availability_df,
                                                                                                path_stilt)

                #If no footprints are found:
                if(stilt_nfp == 0):

                    #Print message:
                    print("\033[0;31;1m No footprints available.\033[0;31;0m\n\n")


                #If footprints are found:
                else:

                    #Set footprints that are equal to zero, equal to NaN:
                    stilt_fp.data[stilt_fp.data==0.0] = np.nan

                    #Call function to plot footprint maps:
                    plot_fp_maps(stilt_fp, stilt_lon, stilt_lat, station_info_df,
                                 title='Aggregated Footrpint (n='+str(stilt_nfp)+')\n'+stilt_title+
                                 '\nStation: '+
                                 station_info_df.name.loc[station_info_df.stationId==icos_stn_id].values[0]+
                                 ' ('+stilt_stn_id + ') ' + stilt_alt + ' m agl ' + '\n['+
                                 stilt_icos_availability_df.locIdent.loc[stilt_icos_availability_df['stiltStationId']==stilt_stn_id].values[0]+
                                 ']',
                                 label='Surface Influence',
                                 unit='[ppm / (\u03BCmol / m\u00B2s)]',
                                 linlog="log10", station_id=icos_stn_id, midpoint=True)


                #If the "citation" checkbox is checked:
                if(Citation):

                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    printmd("<sub>"+'STILT Model Results: https://www.icos-cp.eu/footprint-tool'+"</sub>")

            
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 station_value=station_dropdown,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False),
                                 Footprint=Checkbox(value=False, description='Footprint', disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))
                                 #color=ColorPicker(concise=False,
                                                   #description='Pick a color',
                                                   #value='#3973ac',
                                                   #disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '10px 2px 2px 2px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left
            

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
